In [1]:
%%capture
%pip install -U 'rockfish[labs]' -f 'https://docs142.rockfish.ai/packages/index.html'

In [2]:
import io
import rockfish as rf
import rockfish.actions as ra

## Fill missing values
1. fill by indicated value
2. fill by previous value
3. fill by next value
4. fill by its mean
5. fill by its median


In [4]:
# create a dataset with missing value
data = b"""\
a,b,c
1,2,3
4,5,6
,7,8
9,0,1
"""

dataset = rf.Dataset.from_csv("nulls", io.BytesIO(data))


### 1. fill missing values by the indicated value

In [5]:
dataset.to_pandas()

,a,b,c
0,1.0,2,3
1,4.0,5,6
2,NaN,7,8
3,9.0,0,1


In [6]:
conn = rf.Connection.local()

In [7]:
fill_value = 42
fill_col = "a"
fill_null = ra.Transform({"function": {"fill_null": [fill_col, fill_value]}})

In [8]:
save = rf.actions.DatasetSave(name="fill_value_dataset")
builder = rf.WorkflowBuilder.local()
builder.add_dataset(dataset)
builder.add_action(fill_null, parents=[dataset])
builder.add_action(save, parents=[fill_null])
workflow = await builder.start(conn)
print(f"Workflow: {workflow.id()}")

Workflow: 855216ab-d5bf-41ca-b8a1-a4d1d0240adf


In [9]:
new_dataset = None
async for sds in workflow.datasets():
    new_dataset = await sds.to_local(conn)
new_dataset.to_pandas()

,a,b,c
0,1,2,3
1,4,5,6
2,42,7,8
3,9,0,1


### 2. fill missing values by its previous value in that column

In [10]:
dataset.to_pandas()

,a,b,c
0,1.0,2,3
1,4.0,5,6
2,NaN,7,8
3,9.0,0,1


In [11]:
fill_col = "a"
fill_null = ra.Transform({"function": {"fill_null_forward": [fill_col]}})

In [12]:
save = rf.actions.DatasetSave(name="fill_null_forward_dataset")
builder = rf.WorkflowBuilder()
builder.add_dataset(dataset)
builder.add_action(fill_null, parents=[dataset])
builder.add_action(save, parents=[fill_null])
workflow = await builder.start(conn)
print(f"Workflow: {workflow.id()}")

Workflow: e6055113-1732-483c-b497-4526da0c3019


In [13]:
new_dataset = None
async for sds in workflow.datasets():
    new_dataset = await sds.to_local(conn)
new_dataset.to_pandas()

,a,b,c
0,1,2,3
1,4,5,6
2,4,7,8
3,9,0,1


### 3. fill missing values by its next value in that column

In [14]:
dataset.to_pandas()

,a,b,c
0,1.0,2,3
1,4.0,5,6
2,NaN,7,8
3,9.0,0,1


In [15]:
fill_col = "a"
fill_null = ra.Transform({"function": {"fill_null_backward": [fill_col]}})

In [16]:
save = rf.actions.DatasetSave(name="fill_null_backward_dataset")
builder = rf.WorkflowBuilder()
builder.add_dataset(dataset)
builder.add_action(fill_null, parents=[dataset])
builder.add_action(save, parents=[fill_null])
workflow = await builder.start(conn)
print(f"Workflow: {workflow.id()}")

Workflow: 06496fe8-0c09-48bd-a022-a3f7a1bd5d39


In [17]:
new_dataset = None
async for sds in workflow.datasets():
    new_dataset = await sds.to_local(conn)
new_dataset.to_pandas()

,a,b,c
0,1,2,3
1,4,5,6
2,9,7,8
3,9,0,1


### 4. fill missing values by its mean value in that column

In [18]:
dataset.to_pandas()

,a,b,c
0,1.0,2,3
1,4.0,5,6
2,NaN,7,8
3,9.0,0,1


In [19]:
fill_col = "a"
fill_method = "mean"
fill_null = ra.Transform({"function": {"fill_null_aggregation": [fill_col, fill_method]}})

In [20]:
save = rf.actions.DatasetSave(name="fill_mean_dataset")
builder = rf.WorkflowBuilder()
builder.add_dataset(dataset)
builder.add_action(fill_null, parents=[dataset])
builder.add_action(save, parents=[fill_null])
workflow = await builder.start(conn)
print(f"Workflow: {workflow.id()}")

Workflow: 75ac7b9b-376e-46d2-9637-09ec86ea4434


In [21]:
new_dataset = None
async for sds in workflow.datasets():
    new_dataset = await sds.to_local(conn)
new_dataset.to_pandas()

,a,b,c
0,1.000000,2,3
1,4.000000,5,6
2,4.666667,7,8
3,9.000000,0,1


### 5. fill missing values by its median value in that column

In [22]:
dataset.to_pandas()

,a,b,c
0,1.0,2,3
1,4.0,5,6
2,NaN,7,8
3,9.0,0,1


In [23]:
fill_col = "a"
fill_method = "median"
fill_null = ra.Transform({"function": {"fill_null_aggregation": [fill_col, fill_method]}})

In [24]:
save = rf.actions.DatasetSave(name="fill_median_dataset")
builder = rf.WorkflowBuilder()
builder.add_dataset(dataset)
builder.add_action(fill_null, parents=[dataset])
builder.add_action(save, parents=[fill_null])
workflow = await builder.start(conn)
print(f"Workflow: {workflow.id()}")

Workflow: 15bbaf3d-70c8-4b08-9fe8-3aed90a7e79c


In [25]:
new_dataset = None
async for sds in workflow.datasets():
    new_dataset = await sds.to_local(conn)
new_dataset.to_pandas()

,a,b,c
0,1.0,2,3
1,4.0,5,6
2,4.0,7,8
3,9.0,0,1


## Append new column for the transformed field

Add new column for the result after filling missing with indicated values and the original column with missing values keeps the same

In [26]:
dataset.to_pandas()

,a,b,c
0,1.0,2,3
1,4.0,5,6
2,NaN,7,8
3,9.0,0,1


In [27]:
fill_value = 42
fill_col = "a"
new_col_name = "new_a"
fill_null = ra.Apply({"function": {"fill_null": [fill_col, fill_value]}, "append_field": new_col_name})

In [28]:
save = ra.DatasetSave({"name": "new_column_filled_dataset"})
builder = rf.WorkflowBuilder()
builder.add_dataset(dataset)
builder.add_action(fill_null, parents=[dataset])
builder.add_action(save, parents=[fill_null])
workflow = await builder.start(conn)

print(f"Workflow: {workflow.id()}")

Workflow: b61a0cd6-96f5-4f77-8990-3da4e22685b6


In [29]:
new_dataset = None
async for sds in workflow.datasets():
    new_dataset = await sds.to_local(conn)
new_dataset.to_pandas()

,a,b,c,new_a
0,1.0,2,3,1
1,4.0,5,6,4
2,NaN,7,8,42
3,9.0,0,1,9
